**Домашнее задание 8**
<br>Цель: разработать систему, которая будет формировать описания покупателей на основе характеристик, представленных в файле.

<br>На входе — CSV-файл с характеристиками покупателей.

<br>На выходе — TXT-файл с описаниями.

<br>Шаги:

<li>Загрузить CSV-файл.
<li>Выполнить парсинг (извлечение атрибутов из структурированного текста).
<li>Преобразовать данные (при необходимости).
<li>Сформировать текстовое описание по шаблону.
<li>Записать в единый TXT-файл.

<br>Пример строки на входе:

<br>ФИО: Allen Miss. Elisabeth Walton
<br>Пол: female
<br>Возраст: 29
<br>Устройство, с которого выполнялась покупка: mobile
<br>Браузер: Chrome
<br>Сумма чека: 885
<br>Регион покупки: St Louis: MO

<br>Пример строки на выходе:

Пользователь Allen Miss. Elisabeth Walton женского пола, 29 лет совершила покупку на 885 у.е. с мобильного браузера Chrome. Регион, из которого совершалась покупка: St Louis: MO.

**Исследуем данные:**
<br> файл 'web_clients_correct.csv' необходимо положить в корень диска d

In [1]:
import pandas as pd
import os

if os.path.exists('d:/web_clients_correct.csv'):
    df = pd.read_csv('d:/web_clients_correct.csv', sep=',')
else:
    print('Такого файла не существует!')

In [2]:
# нулевых значений нет, 'возраст' - все числа с плавающей запятой, 'стоимость' - все числа целые
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         915 non-null    object 
 1   device_type  915 non-null    object 
 2   browser      915 non-null    object 
 3   sex          915 non-null    object 
 4   age          915 non-null    float64
 5   bill         915 non-null    int64  
 6   region       915 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 50.2+ KB


In [3]:
# список полей данных
df.columns.tolist()

['name', 'device_type', 'browser', 'sex', 'age', 'bill', 'region']

In [4]:
# количество уникальных значений каждого поля
# из важного - 'пол' - 2 значения, 'тип устройства' - 4 значения
for el in df.columns.tolist():
    print(f'{el}: {df[el].nunique()}')

name: 914
device_type: 4
browser: 4
sex: 2
age: 61
bill: 679
region: 367


In [5]:
# статистика по числовым полям - аномалий нет
df.describe()

,age,bill
count,915.000000,915.000000
mean,36.500186,774.320219
std,12.241564,413.293281
min,0.670000,33.000000
25%,27.000000,418.500000
50%,33.000000,778.000000
75%,45.000000,1138.500000
max,80.000000,1500.000000


In [6]:
# возможные типы устройств
df['device_type'].value_counts()

tablet     238
mobile     235
laptop     221
desktop    221
Name: device_type, dtype: int64

**Основная часть**

In [7]:
# создадим класс Покупка
class purchase():
    # инициализация экзеемпляра класса
    def __init__(self, name, device_type, browser, sex, age, bill, region):
        self.name = name
        self.device_type = device_type
        self.browser = browser
        self.sex = sex
        self.age = age
        self.bill = bill
        self.region = region
        # значения атрибутов экземпляра, которые зависят от входных данных
        if sex == 'male':
            self.sex_text = 'мужского'
            self.action = 'совершил'
        else:
            self.sex_text = 'женского'
            self.action = 'совершила'
            
        if device_type == 'tablet':
            self.device_text = 'планшетного'
        elif device_type == 'mobile':
            self.device_text = 'мобильного'
        elif device_type == 'laptop':
            self.device_text = 'лэптопного'
        else:
            self.device_text = 'десктопного'
        
    # функция для формирования текста согласно задания
    def print_text(self):
        return (f'Пользователь {self.name} {self.sex_text} пола, {self.age} лет, {self.action} покупку на {self.bill} у.е. с {self.device_text} браузера {self.browser}. Регион, из которого совершалась покупка: {self.region}')

In [8]:
# используем контекстный менеджер для работы с файлами
with open("d:\\web_clients_correct.csv", "r") as file1, \
open("d:\\result.txt", "w", encoding = 'utf-8') as file2:
    content = file1.readline() # пропускаем заголовок
    while True:
        content = file1.readline().split(',') # читаем строку в список с разделителем запятая
        if content == ['']: # если считали пустую строку, то закончить чтение
            break        
        content[-1] = content[-1].rstrip('\n') # удаляем символ переноса у последнего элемента списка
        # создаем экземпляр класса Покупка, используем для инициализации считанные ранее данные
        new_purchase = purchase(content[0], content[1], content[2], content[3], content[4], content[5], content[6])
        print(new_purchase.print_text(), file=file2) # выводим на печать в файл текст